In [ ]:
#CAUTION: Takes a long time to load.
install.packages("themis")

In [ ]:
#load libraries
library(tidyverse)
library(repr)
library(tidymodels)
library(themis)
options(repr.matrix.max.rows = 10)

In [ ]:
#Add columns
unscaled_data <- read_csv("pulsar_data.csv", 
                        col_names = c("mean_integrated_profile", 
                                      "stand_dev_integrated_profile", 
                                      "exc_kurtosis_integrated_profile", 
                                      "skew_integrated_profile",
                                      "mean_dmsnr", 
                                      "stand_dev_dmsnr", 
                                      "exc_kurtosis_dmsnr", 
                                      "skew_dmsnr", "class")) 

unscaled_data <- unscaled_data |>
    mutate(class = as_factor(class))
#unscaled_data 

#Class proportions in pulsar dataset (Imbalanced)
num_obs <- nrow(unscaled_data)
pulsar_proportions <- unscaled_data |>
    group_by(class) |>
    summarize(n = n()) |>
    mutate(percent = 100*n/nrow(unscaled_data))
pulsar_proportions

#Scale data and Upsample to balance data
pulsar_recipe <- recipe(class ~ exc_kurtosis_integrated_profile + skew_dmsnr + stand_dev_integrated_profile + 
                        mean_dmsnr + skew_integrated_profile + exc_kurtosis_dmsnr, data = unscaled_data) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors()) |>
    step_upsample(class, over_ratio = 1, skip = FALSE) |>
    prep()

standardized_pulsar <- bake(pulsar_recipe, unscaled_data)
standardized_pulsar

#check proportions of upsampled data
new_pulsar_proportions <- standardized_pulsar |>
    group_by(class) |>
    summarize(n = n()) |>
    mutate(percent = 100*n/nrow(standardized_pulsar))
new_pulsar_proportions

In [ ]:
set.seed(1)
pulsar_split <- initial_split(standardized_pulsar, prop = 0.75, strata = class)
pulsar_train <- training(pulsar_split)
pulsar_test <- testing(pulsar_split) 

pulsar_recipe <- recipe(class ~ exc_kurtosis_integrated_profile + skew_dmsnr + stand_dev_integrated_profile + 
                        mean_dmsnr + skew_integrated_profile + exc_kurtosis_dmsnr, data = pulsar_train)

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 3) |>
    set_engine("kknn") |>
    set_mode("classification")

pulsar_fit <- workflow() |>
    add_recipe(pulsar_recipe) |>
    add_model(knn_spec) |>
    fit(data = pulsar_train)
pulsar_fit

pulsar_test_predictions <- predict(pulsar_fit, pulsar_test) |>
    bind_cols(pulsar_test)
pulsar_test_predictions

pulsar_prediction_accuracy <- pulsar_test_predictions |>
    metrics(truth = class, estimate = .pred_class) |>
    filter(.metric == "accuracy")
pulsar_prediction_accuracy

confusion <- pulsar_test_predictions |>
    conf_mat(truth = class, estimate = .pred_class)
confusion

In [ ]:
#10 fold cross validation
pulsar_vfold <- vfold_cv(pulsar_train, v = 10, strata = class)

pulsar_resample_fit <- workflow() |>
       add_recipe(pulsar_recipe) |>
       add_model(knn_spec) |>
       fit_resamples(resamples = pulsar_vfold)

pulsar_metrics <- collect_metrics(pulsar_resample_fit)
pulsar_metrics

In [ ]:
#CAUTION: Takes a long time to load.
knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")
#knn_tune

k_vals <- tibble(neighbors = seq(from = 1, to = 50, by = 3))

knn_results <- workflow() |>
               add_recipe(pulsar_recipe) |>
               add_model(knn_tune) |>
               tune_grid(resamples = pulsar_vfold, grid = k_vals) |>
               collect_metrics()
#knn_results

accuracies <- knn_results |> 
       filter(.metric == "accuracy")

accuracy_versus_k <- ggplot(accuracies, aes(x = neighbors, y = mean))+
       geom_point() +
       geom_line() +
       labs(x = "Neighbors", y = "Accuracy Estimate")  
accuracy_versus_k


In [ ]:
#we can have a separate cell for each combination of predictors that we try
#pick one K value to use for all the combinations, to avoid having to find the best K each time (which could vary)
#we could call the predictor combinations combo1, combo2, etc.
#compare their accuracies, choose the most accurate set of predictors
#Read Chapter 6.8 on selecting good predictors.

In [ ]:
knn_spec_final <- nearest_neighbor(weight_func = "rectangular", neighbors = 5) |>
                    set_engine("kknn") |>
                    set_mode("classification")

pulsar_fit_final <- workflow() |>
        add_recipe(pulsar_recipe) |>
        add_model(knn_spec_final) |>
        fit(data = pulsar_train)

pulsar_test_predictions_final <-  predict(pulsar_fit_final, data = pulsar_test) |>
        bind_cols(pulsar_test)

pulsar_test_predictions                

Visualization of our classifier's accuracy:

In [ ]:
pulsar_accuracy_plot_1 <- ggplot() +
    geom_point(data = pulsar_test_predictions, mapping = aes(x = exc_kurtosis_integrated_profile, 
                                                              y = skew_dmsnr, colour = class)) +
    geom_point(data = pulsar_test_predictions, mapping = aes(exc_kurtosis_integrated_profile,
                                                             y = skew_dmsnr, colour = .pred_class), 
                                                             alpha = 0.02, size = 5)
    labs(x = "Scaled Excess Kurtosis - Integrated Profile", y = "Scaled Skewness of the DMSNR curve", colour = "Class") +
    theme(text = element_text(size = 20))
pulsar_accuracy_plot_1

#pulsar_accuracy_plot_2 <- do the same thing here (and for plot 3), but use 2 of the other predictors

#pulsar_accuracy_plot_3 <-